In [1]:
""" _summary_

_extended_summary_
"""
import zipfile
import os
import logging
import csv
import gzip
from datetime import datetime

In [3]:
# Descomprime el archivo
with zipfile.ZipFile("T_MASTER_CORD.zip", 'r') as zip_ref:
    zip_ref.extractall()
csvfile = os.path.join(os.getcwd(), zip_ref.namelist()[0])
logging.info("Archivo %s descomprimido", csvfile)

In [4]:
csvfile

'/home/inspired/data-science-on-gcp/04_streaming/design/T_MASTER_CORD.csv'

In [ ]:
def descomprimir_archivo(archivo_zip, directorio=None):
    """Descomprime un archivo ZIP en el directorio especificado.

    Args:
        archivo_zip (str): Ruta al archivo ZIP.
        directorio (str, optional): Ruta al directorio de destino
        (opcional, si no se especifica, se usa el directorio actual).

    Returns:
        str: Ruta al archivo CSV descomprimido.
    """

    with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
        if directorio:
            zip_ref.extractall(directorio)
        else:
            zip_ref.extractall()
        archivo_csv = os.path.join(
            directorio if directorio else os.getcwd(), zip_ref.namelist()[0])
        logging.info("Archivo %s descomprimido", archivo_csv)
        return archivo_csv

In [5]:
# Leemos el CSV con el módulo csv, a una lista
data = []
try:
    with open(csvfile, 'r', encoding="utf-8") as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)  # Leer la primera fila como encabezado
        for row in csv_reader:
            data.append(row)
except Exception as e:
    logging.error("Error al leer el archivo CSV: %s", str(e))

In [ ]:
def leer_csv(archivo_csv):
    """Lee un archivo CSV y devuelve los datos como una lista de listas y el encabezado.

    Args:
        archivo_csv (str): Ruta al archivo CSV.

    Returns:
        tuple: Un tuple que contiene dos elementos:
        - data (list): Lista de listas que representan las filas y columnas del archivo CSV.
        - header (list): Lista que representa la primera fila (encabezado) del archivo CSV.
    """

    data = []
    try:
        with open(archivo_csv, 'r', encoding="utf-8") as file:
            csv_reader = csv.reader(file)
            header = next(csv_reader)  # Leer la primera fila como encabezado
            for row in csv_reader:
                data.append(row)
    except Exception as e:
        logging.error("Error al leer el archivo CSV: %s", str(e))
        data = []
        header = []  # Set data and header to empty lists in case of error

    return data, header

In [ ]:
# Transformamos al formato de entrada, mediante la creación de una función
def transformar_fecha(fila, columna):
    """transformar_fecha _summary_

    _extended_summary_

    Arguments:
        row -- _description_
        columna -- _description_
    """

    date_format = "%m/%d/%Y %I:%M:%S %p"
    try:
        if fila[columna]:
            fila[columna] = datetime.strptime(
                fila[columna], date_format).strftime('%Y-%m-%d')
            logging.info(
                "Fecha en la columna %s formateada correctamente",
                header[columna]
            )
        else:
            logging.warning(
                "La fecha en la columna %s está vacía", header[columna])
    except (IndexError, ValueError) as eerr:
        logging.warning(
            "No se pudo formatear la fecha en la columna %s: %s", header[columna], str(eerr))

In [ ]:
# Aplicamos la función transformar_fecha
for row in data:
    transformar_fecha(row, header.index('AIRPORT_START_DATE'))
    transformar_fecha(row, header.index('AIRPORT_THRU_DATE'))

In [11]:
def transformar_fechas(data, header):
    """Transforma las fechas en las columnas especificadas de un conjunto de datos.

    Args:
        data (list): Lista de listas que representan las filas y columnas del archivo CSV.
        header (list): Lista que representa la primera fila (encabezado) del archivo CSV.

    Returns:
        None
    """

    # Por inspección previa conocemos el formato de origen
    date_format = "%m/%d/%Y %I:%M:%S %p"
    # Conocemos cuales son las columnas con fecha
    columnas_fecha = ['AIRPORT_START_DATE', 'AIRPORT_THRU_DATE']

    for row in data:
        for columna in columnas_fecha:
            indice_columna = header.index(columna)
            try:
                if row[indice_columna]:
                    row[indice_columna] = datetime.strptime(
                        row[indice_columna], date_format).strftime('%Y-%m-%d')
                    logging.info(
                        "Fecha en la columna %s formateada correctamente", columna)
                else:
                    logging.warning(
                        "La fecha en la columna %s está vacía", columna)
            except (IndexError, ValueError) as e:
                logging.warning(
                    "No se pudo formatear la fecha en la columna %s: %s", columna, str(e))

In [12]:

transformar_fechas(data, header)

In [14]:
# Escribimos a csv sin comprimir para revisión rápida
csv_output_file = "airports_2024.csv"
try:
    with open(csv_output_file, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(header)
        csv_writer.writerows(data)
    logging.info("Archivo CSV creado con éxito: %s", csv_output_file)
except Exception as e:
    logging.error("Error al escribir el archivo CSV: %s", str(e))

In [ ]:
def escribir_csv(data, header):
    """Escribe una lista de datos a un archivo CSV y retorna el archivo.

    Args:
        data (list): Lista de listas que representan las filas y columnas del archivo CSV.
        header (list): Lista que representa la primera fila (encabezado) del archivo CSV.
        output_file (str): Ruta al archivo CSV de salida.

    Returns:
        str: Ruta al archivo CSV creado.
    """

    csv_output_file = "airports_2024.csv"

    try:
        with open(csv_output_file, 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)
            csv_writer.writerows(data)
        logging.info("Archivo CSV creado con éxito: %s", csv_output_file)
        return csv_output_file
    except Exception as e:
        logging.error("Error al escribir el archivo CSV: %s", str(e))
        return None

In [15]:
# Escribimos a csv.gz
gz_output_file = "airports_2024.csv.gz"
try:
    with gzip.open(gz_output_file, 'wt', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(header)
        csv_writer.writerows(data)
    logging.info("Archivo CSV comprimido creado con éxito: %s", gz_output_file)
except Exception as e:
    logging.error("Error al escribir el archivo CSV comprimido: %s", str(e))

In [ ]:
def escribir_csv_gz(data, header):
    """Escribe una lista de datos a un archivo CSV comprimido con gzip y retorna el archivo.

    Args:
        data (list): Lista de listas que representan las filas y columnas del archivo CSV.
        header (list): Lista que representa la primera fila (encabezado) del archivo CSV.
        gz_output_file (str): Ruta al archivo CSV comprimido de salida.

    Returns:
        str: Ruta al archivo CSV comprimido creado, o None en caso de error.
    """

    gz_output_file = "airports_2024.csv.gz"
    try:
        with gzip.open(gz_output_file, 'wt', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)
            csv_writer.writerows(data)
        # Return the output file path on success
        return gz_output_file
    except Exception as e:
        logging.error(
            "Error al escribir el archivo CSV comprimido: %s", str(e))
        return None

In [ ]:
# Eliminar archivo csv temporal
os.remove(csvfile)